In [1]:
import numpy as np

np.set_printoptions(
    suppress=True,
    linewidth=180
)

# Board generation

In [2]:
def generate_board(num_actions):
    import random
    actions = [
        {
            "id": i,
            "attackerCost": random.randint(1, 5),
            "defenderCost": random.randint(1, 5),
            "attackerProb": round(random.uniform(0.5, 0.98),2),
            "defenderProb": round(random.uniform(0.5, 0.98),2),
            "severity": random.randint(1,5),
        }
        for i in range(num_actions)
    ]
    for v in actions:
        v["risk"] = round(v["severity"] ** 2 * v["attackerProb"] * (1-v["defenderProb"]) * (v["defenderCost"] / v["attackerCost"]),2)
    return actions

# Utility matrix generation

In [3]:
def powerset(s):
    import itertools
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1, len(s)+1))

In [4]:
def get_utility(attackerMoves, defenderMoves):
    defenderLabels = set([m["id"] for m in defenderMoves])
    return sum([
        move["severity"]**2 * move["attackerProb"]
        if move["id"] not in defenderLabels else
        move["severity"]**2 * move["attackerProb"] * (1-move["defenderProb"])
        for move in attackerMoves
    ])

In [5]:
def validate_cost(moves, cost_func, max_cost):
    return sum([cost_func(m) for m in moves]) <= max_cost

In [6]:
def generate_utility_matrices(
    board,
    attacker_cost_available,
    defender_cost_available,
    defender_knows_attacker_cost
):
    all_move_sets = list(powerset(board))
    attacker_utility_matrix = np.zeros((len(all_move_sets),len(all_move_sets)))
    defender_utility_matrix = np.zeros((len(all_move_sets),len(all_move_sets)))
    print(f"Found {len(all_move_sets)} move combinations ({len(all_move_sets) ** 2:,} for both)")

    for i, attacker_choice in enumerate(all_move_sets):
        for j, defender_choice in enumerate(all_move_sets):
            # if the attacker moveset is invalid
            if not validate_cost(attacker_choice, lambda x: x["attackerCost"], attacker_cost_available):
                # terrible attacker utility
                attacker_utility_matrix[i][j] = -999
                # if the defender moveset is also invalid
                if not validate_cost(defender_choice, lambda x: x["defenderCost"], defender_cost_available):
                    # terrible defender utility
                    defender_utility_matrix[i][j] = -999
                else:
                    # utility is dependent on whether the defender knows the attacker max cost or not
                    defender_utility_matrix[i][j] = -999 if defender_knows_attacker_cost else -get_utility(attacker_choice, defender_choice)
            else:
                # normal attacker utility otherwise
                attacker_utility_matrix[i][j] = get_utility(attacker_choice, defender_choice)
                # if defender moveset is invalid
                if not validate_cost(defender_choice, lambda x: x["defenderCost"], defender_cost_available):
                    # terrible defender utility
                    defender_utility_matrix[i][j] = -999
                else:
                    # normal defender utility otherwise
                    defender_utility_matrix[i][j] = -get_utility(attacker_choice, defender_choice)
    return attacker_utility_matrix.transpose(), defender_utility_matrix

# Optimization problem solver

In [7]:
def get_optimized_solution(utility_matrix):
    import scipy.optimize
    num_options = len(utility_matrix)

    c = np.zeros(num_options+1)
    c[0] = -1

    A_ub = np.ones((num_options, num_options+1))
    A_ub[:,1:] = utility_matrix*-1
    b_ub = np.zeros(num_options)

    A_eq = np.ones((1, num_options+1))
    A_eq[0][0] = 0
    b_eq = 1

    lb = np.zeros(num_options+1)
    lb[0] = -10000
    ub = np.ones(num_options+1)
    ub[0] = 10000
    bounds = np.asarray([lb, ub]).transpose()

    result = scipy.optimize.linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq,
        bounds=bounds,
        # options = {
        #     "tol": 0.001
        #     # "autoscale": True
        # }
        # method="simplex",
        method="highs",
        # method="interior-point",
        # options={"presolve":False},
        # callback = lambda x: zz.append(x.x)
    )
    if result.success:
        return result.x[1:]
    else:
        print(result)
        raise Exception("Couldn't find solution")

In [8]:
def get_average_solution(utility_matrix):
    return np.ones(len(utility_matrix)) / len(utility_matrix)

# Investigation

In [9]:
board = generate_board(11)
attacker_utility, defender_utility = generate_utility_matrices(
    board=board,
    attacker_cost_available=10,
    defender_cost_available=10,
    defender_knows_attacker_cost=False
)
attacker_optimal = get_optimized_solution(attacker_utility)
print("maximum attacker utility", sum(attacker_utility @ attacker_optimal))
print("average attacker utility", sum(attacker_utility @ get_average_solution(attacker_utility)))

defender_optimal = get_optimized_solution(defender_utility)
print("expected defender utility", sum(defender_utility @ defender_optimal))
print("average defender utility", sum(defender_utility @ get_average_solution(defender_utility)))

Found 2047 move combinations (4,190,209 for both)
maximum attacker utility 64231.239600000335
average attacker utility -1859576.3586790452
expected defender utility -48879.41119999997
average defender utility -1739594.9447151965


In [ ]:
import scipy.io
scipy.io.savemat("../matlab/utility.mat", {
    "attacker_utility": attacker_utility,
    "defender_utility": defender_utility,
})